## 作業六

In [1]:
import pandas
finance = pandas.read_excel('https://raw.githubusercontent.com/ywchiu/tibame_tm/master/data/finance_article.xlsx')

In [2]:
finance.head()

,url,title,content,tag
0,https://www.ptt.cc/bbs/Loan/M.1586506724.A.914...,[問題] 新北原屋貸款,[抵押房屋資訊]\n地點：新北市永和\n房屋形態：透天五樓\n房屋種類：自有住宅\n房屋/土...,Loan
1,https://www.ptt.cc/bbs/Loan/M.1586449508.A.618...,信貸 (台積電),1. 信貸金額：80-200萬\n2. 還款期限：7年\n3. 貸款用途：投資理財\n4. ...,Loan
2,https://www.ptt.cc/bbs/Stock/M.1586573726.A.DB...,[新聞] 瑞幸咖啡遭遇做空 疫情危機下中概股受到的放大鏡式狙擊,1.原文連結：\n\n2.原文內容：\n林祖偉／BBC中文\n\n瑞幸咖啡有超過4500間分...,Stock
3,https://www.ptt.cc/bbs/Stock/M.1586582519.A.AE...,[新聞] 14檔業績四升 法人搶,1.原文連結：\n\n2.原文內容：14檔業績四升 法人搶\n2020-04-11 00:1...,Stock
4,https://www.ptt.cc/bbs/Stock/M.1586572758.A.8B...,[新聞] 突變酵素 塑膠瓶幾小時內能回收,1.原文連結：\n\n2.原文內容：\n科學家創造出一種突變細菌酵素，能在幾小時分解塑膠瓶進...,Stock


In [3]:
import jieba
corpus = []
tags = []
for idx, rec in finance.iterrows():
  corpus.append(' '.join(jieba.cut(rec['content'])))
  tags.append(rec['tag'])

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.959 seconds.
Prefix dict has been built successfully.


In [6]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vectorizer = TfidfVectorizer()
tfidf = vectorizer.fit_transform(corpus)

In [7]:
X = tfidf
y = tags

In [8]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(X,y, test_size = 0.2, random_state = 42)

In [9]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB(alpha = 0.01)
clf.fit(train_x, train_y)

MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True)

In [11]:
pred_y = clf.predict(test_x)

In [12]:
from sklearn.metrics import accuracy_score
accuracy_score(test_y, pred_y)

1.0

In [14]:
from sklearn.metrics import confusion_matrix
print(clf.classes_)
confusion_matrix(test_y, pred_y)

['Loan' 'Stock' 'creditcard']


array([[24,  0,  0],
       [ 0, 17,  0],
       [ 0,  0, 19]])

## Yahoo 奇摩爬蟲

In [16]:
import requests
res = requests.get('https://movies.yahoo.com.tw/movieinfo_review.html/id=8779?sort=update_ts&order=desc&page=2')


In [17]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(res.text, 'lxml')

In [19]:
#soup

In [29]:
movie_ary = []
for li in soup.select('.usercom_list li'):
  stars = li.select_one('input[name=score]').get('value')
  comments = li.select('span')[2].text
  movie_ary.append({'comments':comments, 'stars':stars})
  #print('====================')

In [30]:
import pandas
moviedf = pandas.DataFrame(movie_ary)

In [31]:
moviedf

,comments,stars
0,為拍而拍 全毀 面目全非 女主選角選成這樣我已經忍著掏錢出來看 結果整體也是爛的一匹,1
1,普普 看到想睡覺 劇情流暢 但看完不是很過癮,3
2,比復仇者聯盟4~好看太多!驚奇隊長~好看~推,5
3,https://www.facebook.com/profile.php?id=100028...,5
4,比預期好看，幾部英雄系列電影裡覺得很推薦的一部,4
5,看片找我 www.gamedisc.cc,5
6,沒有想像中好看，但交代了一些以前復仇者聯盟系列沒有的細節，總體還行,4
7,真是很乏味無趣的一部戲.超級失望~,1
8,女角給1星。其它不是很有趣。\r\n內容適合國中小學生。,1
9,對女超人無抵抗～正妹！力挺哈哈,4


In [32]:
def convertRate(stars):
  if int(stars) >= 4:
    return 'good'
  elif int(stars) == 3:
    return 'soso'
  else:
    return 'bad'

In [34]:
moviedf['status'] = moviedf['stars'].map(convertRate)

In [35]:
moviedf

,comments,stars,status
0,為拍而拍 全毀 面目全非 女主選角選成這樣我已經忍著掏錢出來看 結果整體也是爛的一匹,1,bad
1,普普 看到想睡覺 劇情流暢 但看完不是很過癮,3,soso
2,比復仇者聯盟4~好看太多!驚奇隊長~好看~推,5,good
3,https://www.facebook.com/profile.php?id=100028...,5,good
4,比預期好看，幾部英雄系列電影裡覺得很推薦的一部,4,good
5,看片找我 www.gamedisc.cc,5,good
6,沒有想像中好看，但交代了一些以前復仇者聯盟系列沒有的細節，總體還行,4,good
7,真是很乏味無趣的一部戲.超級失望~,1,bad
8,女角給1星。其它不是很有趣。\r\n內容適合國中小學生。,1,bad
9,對女超人無抵抗～正妹！力挺哈哈,4,good


In [37]:
import pandas
movies  = pandas.read_excel('https://raw.githubusercontent.com/ywchiu/tibame_tm/master/data/yahoo_movie.xlsx', index_col = 0)
movies.head()

,content,stars,title,status
0,"不知道耶! 看完整個無感\r\n有種覺得就是女兒""給蕭""害死了老爸\r\n然後...既然可以...",3,古墓奇兵,soso
1,很好看的動作片，不會浪費錢跟時間。很久沒有這樣的探險片。可說是女版的印第安那瓊。女主角跟爸爸...,5,古墓奇兵,good
2,說這個不好看的話，那我還真不知道，還有什麼片是您可以去看得了。電影好看，但話說羅拉有裝可以撿...,5,古墓奇兵,good
3,父女重逢真的很讓人感動，五顆星。,5,古墓奇兵,good
4,劇情雖然老套，但仍拍出新意，古墓能殺人的方式不就是機關和毒，要求亂七八糟的觀眾，你看喪尸片看多了。,4,古墓奇兵,good


In [39]:
movies['title'].value_counts()

最終騎士       452
奧創紀元       281
攻殼機動隊      173
最後的絕地武士    117
古墓奇兵        89
Name: title, dtype: int64

In [41]:
import jieba
corpus = []
tags = []
for idx, rec in movies[movies['status'].isin(['good', 'bad'])].iterrows():
  corpus.append(' '.join(jieba.cut(rec.get('content'))))
  tags.append(rec.get('status'))

In [42]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)
X

<980x5900 sparse matrix of type '<class 'numpy.float64'>'
	with 16897 stored elements in Compressed Sparse Row format>

In [43]:
y = tags

In [44]:
from sklearn.model_selection import train_test_split
train_X, test_X, train_Y, test_Y = train_test_split(X, y, test_size = 0.3, random_state =42)

In [45]:
train_X.shape

(686, 5900)

In [46]:
test_X.shape

(294, 5900)

In [47]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB(alpha = 0.01)
clf.fit(train_X, train_Y)

MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True)

In [48]:
predicted = clf.predict(test_X)

In [49]:
from sklearn.metrics import accuracy_score, confusion_matrix
accuracy_score(test_Y, predicted)

0.7653061224489796

In [51]:
print(clf.classes_)
confusion_matrix(test_Y, predicted)

['bad' 'good']


array([[ 83,  34],
       [ 35, 142]])

In [53]:
clf.feature_count_.shape

(2, 5900)

In [55]:
#vectorizer.get_feature_names()

In [56]:
features_ary = []
for word, bad_count, good_count in zip(vectorizer.get_feature_names(), clf.feature_count_[0], clf.feature_count_[1]):
  dic = {}
  dic['good_ratio'] = good_count/(bad_count + 1)
  dic['bad_ratio'] = bad_count / (good_count + 1)
  dic['good_cnt'] = good_count
  dic['bad_cnt'] = bad_count
  dic['word'] = word
  features_ary.append(dic)

In [57]:
import pandas
features_df = pandas.DataFrame(features_ary)

In [60]:
features_df.sort_values('good_ratio', ascending=False).head(10)

,good_ratio,bad_ratio,good_cnt,bad_cnt,word
5387,4.817935,0.035680,6.025683,0.250677,還不錯
1998,4.779283,0.171791,31.293674,5.547776,好看
683,4.431739,0.070624,6.906312,0.558375,不錯
5722,3.213449,0.118747,5.813320,0.809059,非常
2265,2.985051,0.000000,2.985051,0.000000,希望
5649,2.788027,0.121061,4.717970,0.692225,雖然
3241,2.642371,0.072733,3.508927,0.327946,有些
3839,2.640637,0.178569,5.889079,1.230174,爽片
1255,2.510567,0.100913,3.701753,0.474469,刺激
678,2.387562,0.113462,3.646203,0.527166,不過


In [61]:
features_df.sort_values('bad_ratio', ascending=False).head(10)

,good_ratio,bad_ratio,good_cnt,bad_cnt,word
4204,0.036470,5.473130,0.294949,7.087424,睡著
3606,0.039703,4.792868,0.284045,6.154257,浪費
4113,0.184471,3.658543,2.643348,13.329343,看到
4206,0.013387,3.540975,0.063817,3.766948,睡覺
3667,0.000000,3.219715,0.000000,3.219715,演什麼
1938,0.148842,3.200874,1.194218,7.023414,失望
3822,0.058056,2.630249,0.248742,3.284501,爛片
1871,0.040038,2.584796,0.160096,2.998612,大爛片
4323,0.059520,2.496003,0.244388,3.105997,第一次
1487,0.139935,2.492822,0.750607,4.363952,只有


In [63]:
s = '真是很乏味無趣的一部戲.超級失望~'
s_seg = ' '.join(jieba.cut(s))
s_vec = vectorizer.transform([s_seg])

In [65]:
clf.predict(s_vec)

array(['bad'], dtype='<U4')